In [1]:
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from pyspark.sql import functions as F
import nltk
import string

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /common/home/vig4/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /common/home/vig4/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
raw_df = spark.read.json('/common/users/shared/cs543_fall22_group3/combined/combined_raw')
raw_df.show()

+--------------------+--------------------+
|        publish_date|       selected_text|
+--------------------+--------------------+
| 2019-07-09 03:01:00|Jeffrey Epstein h...|
| 2019-07-09 14:50:00|7 outdoor brands ...|
|            Columbia|                null|
|Shop Columbia out...| Columbia has hel...|
|            Cotopaxi|                null|
|Shop outdoor gear...| you'll find clea...|
|                 REI|                null|
|Shop outdoor gear...|000 outdoor space...|
|         Hydro Flask|                null|
|Shop Hydro Flask ...| ensure drinkable...|
|      United by Blue|                null|
|Shop outdoor gear...|                null|
|           Patagonia|                null|
|Shop Patagonia ou...|                null|
|      The North Face|                null|
|Shop The North Fa...|"" which connects...|
|Subscribe to our ...|                null|
| Find all the bes...|                null|
| Disclosure: This...|                null|
| 2019-07-09 20:26:37|Joe Manchi

In [3]:
raw_df = raw_df.select('selected_text').na.drop()
raw_df.show()

+--------------------+
|       selected_text|
+--------------------+
|Jeffrey Epstein h...|
|7 outdoor brands ...|
| Columbia has hel...|
| you'll find clea...|
|000 outdoor space...|
| ensure drinkable...|
|"" which connects...|
|Joe Manchin intro...|
|Artist creates fu...|
|A rare Pokémon ca...|
| Aramex told Poke...|
|Save on Google sm...|
|Lady Gaga launche...|
| I was just Born ...|
| 2019—Vic ✨ (@Vom...|
|How to add your b...|
|Lady Gaga to laun...|
|Judge ruled DOJ c...|
|How to get out of...|
|Sales and Deals o...|
+--------------------+
only showing top 20 rows



In [4]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

# Remove punctuation, stopwords, and lowercase the letters
def remove_stopwords(line):
    tokens = word_tokenize(line)
    remove_stopwords = [lemmatize_stemming(t.lower()) for t in tokens if not t in STOPWORDS and t not in string.punctuation and len(t) > 2]
    return ','.join(remove_stopwords)

In [5]:
remove_stopwords_udf = F.udf(lambda z: remove_stopwords(z))
processed_df = raw_df.withColumn('cleaned_text', remove_stopwords_udf(F.col('selected_text')))

In [6]:
processed_df.printSchema()

root
 |-- selected_text: string (nullable = true)
 |-- cleaned_text: string (nullable = true)



In [ ]:
processed_df.write.mode('Overwrite').json('/common/users/shared/cs543_fall22_group3/combined/combined_processed')

In [ ]:
processed_df_2000 = processed_df.take(2000)

for i in range(1000, 1005):
    print(f'Raw text: {processed_df_2000[i].selected_text}')
    print(f'Cleaned text: {processed_df_2000[i].cleaned_text}')
    print()